# Izvještaj chatbot zadatak RBA

##### Tomislav Žiger
 

In [2]:
import requests
from models import PromptRequest, PromptResponse
import os
from dotenv import load_dotenv
from util import ChatbotService, DataHandler


U util file-u sam složio klasu ChatbotService koji olakšava pozive na backend i klasu DataHandler s kojom manipuliram svojim testnim podacima


In [3]:
service = ChatbotService(use_local=True)
data_handler = DataHandler()

Ovdje sam dodao samo malu metodu koja poziva service za svaki x i vraća točnost

In [4]:
def get_accuracy(x, y):
    count = 0
    for message, intent in zip(x, y):
        pred = service.get_intent(message)
        if pred == intent:
            count += 1
    return count / len(x)

Prvo sam napravio možda malo preambiciozni testni skup podataka u fileu "dataset_test_hard.csv" koji je bio pretežak za model

In [5]:
x,y = data_handler.get_test_data_hard_xy()
accuracy = get_accuracy(x,y)
print("Točnost na testnom skupu dataset_test_hard.csv:",accuracy * 100, "%")


Točnost na testnom skupu dataset_test_hard.csv: 22.22222222222222 %


Tako da sam sljedeće napravio jednostavniji testni skup dataset_test_easy.csv i na njemu dobio bolji rezultat

In [6]:
x,y = data_handler.get_test_data_easy_xy()
accuracy = get_accuracy(x,y)
print("Točnost na testnom skupu dataset_test_easy.csv:",accuracy * 100, "%")

Točnost na testnom skupu dataset_test_easy.csv: 77.27272727272727 %


Pa pogledajmo detaljnije greške

In [7]:
x,y = data_handler.get_test_data_easy_xy()
count = 1
for message, intent in zip(x, y):
    pred = service.get_intent(message)
    
    if pred != intent:
        full_response = service.get_full_response(message)
        print("Greška broj:", count)
        print("Krivo klasificirana poruka:", message)
        print("Predikcija:", pred)
        print("Točna vrijednost:", intent)
        print("Vjerojatnosti: ", full_response.probs)
        count += 1

Greška broj: 1
Krivo klasificirana poruka: Gdje se nalaze toaleti?
Predikcija: adresa
Točna vrijednost: toaleti
Vjerojatnosti:  {'adresa': 0.1419696399559988, 'clanstvo': 0.08222454052391313, 'danas_izlozbe': 0.08030585846595743, 'kafic': 0.08580891304156278, 'kontakt': 0.08252249016131535, 'parking': 0.08861330434977256, 'pristupacnost': 0.07959830415871569, 'radno_vrijeme': 0.11908764652662168, 'toaleti': 0.12215527900085106, 'ulaznice': 0.11771402381529152}
Greška broj: 2
Krivo klasificirana poruka: Jel moguće doći sa invalidskim kolicima?
Predikcija: radno_vrijeme
Točna vrijednost: pristupacnost
Vjerojatnosti:  {'adresa': 0.0917178254017818, 'clanstvo': 0.0920821290837714, 'danas_izlozbe': 0.08978510674392985, 'kafic': 0.0881898149185537, 'kontakt': 0.0926200379947291, 'parking': 0.08937218217730908, 'pristupacnost': 0.0888441800291408, 'radno_vrijeme': 0.13998738233965327, 'toaleti': 0.08943957787117333, 'ulaznice': 0.13796176343995759}
Greška broj: 3
Krivo klasificirana poruka: I

### Ovdje je ispis iz gornje ćelije, samo što sam dodao ispod zašto mislim da je došlo do greške
<br>

#### Greška broj: 1
Krivo klasificirana poruka: Gdje se nalaze toaleti?
<br>
Predikcija: adresa
<br>
Točna vrijednost: toaleti
<br>
Vjerojatnosti:  {'adresa': 0.1419696399559988, 'clanstvo': 0.08222454052391313, 'danas_izlozbe': 0.08030585846595743, 'kafic': 0.08580891304156278, 'kontakt': 0.08252249016131535, 'parking': 0.08861330434977256, 'pristupacnost': 0.07959830415871569, 'radno_vrijeme': 0.11908764652662168, 'toaleti': 0.12215527900085106, 'ulaznice': 0.11771402381529152}
<br>

#### Objašnjenje:
<span style="color:green">Vidimo po vjerojatnostima da je za malo pogriješio model 0.14 je za adresa, a na drugom mjestu sa 0.12222 vjerojatnosti je toaleti. Pretpostavljam da je adresa dobila bolju vjerojatnost jer je u train setu "Gdje se nalazite?" označeno s "adresa" i onda je model zbog prvih dvije riječi koje su potpuno identične i treća riječ koja je skoro ista odlučio u korist adresa iako imamo čak riječ toaleti u poruci </span>
#### Greška broj: 2
Krivo klasificirana poruka: Jel moguće doći sa invalidskim kolicima?
<br>
Predikcija: radno_vrijeme
<br>
Točna vrijednost: pristupacnost
<br>
Vjerojatnosti:  {'adresa': 0.0917178254017818, 'clanstvo': 0.0920821290837714, 'danas_izlozbe': 0.08978510674392985, 'kafic': 0.0881898149185537, 'kontakt': 0.0926200379947291, 'parking': 0.08937218217730908, 'pristupacnost': 0.0888441800291408, 'radno_vrijeme': 0.13998738233965327, 'toaleti': 0.08943957787117333, 'ulaznice': 0.13796176343995759}
<br>


#### Objašnjenje:
<span style="color:green">Ovo je teško promašio model, vidimo da na drugom mjestu je "ulaznice", a tocan odgovor pristupacnost je najslabija vjerojatnost, mislim da zato jer je u train skupu drugačiji padež "invalidska kolica", a ovdje "sa invalidskim kolicima" da je model krivo shvatio </span>
<br>



#### Greška broj: 3
Krivo klasificirana poruka: Imate li lift?
<br>
Predikcija: ulaznice
<br>
Točna vrijednost: pristupacnost
<br>
Vjerojatnosti:  {'adresa': 0.07567706634182851, 'clanstvo': 0.08562988677825845, 'danas_izlozbe': 0.07421979703557299, 'kafic': 0.12963790391633928, 'kontakt': 0.07631051332448102, 'parking': 0.08380015758772184, 'pristupacnost': 0.1400870921657484, 'radno_vrijeme': 0.10733536600778602, 'toaleti': 0.07406719583689417, 'ulaznice': 0.15323502100536926}
<br>
#### Objašnjenje:
<span style="color:green">Ovo sam i očekivao da će pogriješiti jer sam namjerno stavio sinonim za dizalo "lift" koji nije u train skupu, tako da bilo bi dobro dodati sinonime u train skup </span>
<br>
#### Greška broj: 4
Krivo klasificirana poruka: Gdi se nalazi parking?
<br>
Predikcija: radno_vrijeme
<br>
Točna vrijednost: parking
<br>
Vjerojatnosti:  {'adresa': 0.11688461378804813, 'clanstvo': 0.08701004687804381, 'danas_izlozbe': 0.0848693493495912, 'kafic': 0.08334055227651328, 'kontakt': 0.08768554581108844, 'parking': 0.1149977730751933, 'pristupacnost': 0.08396727206710977, 'radno_vrijeme': 0.12942460384374146, 'toaleti': 0.08440373531813584, 'ulaznice': 0.12741650759253478}
<br>


#### Objašnjenje:
<span style="color:green">Vidimo po vjerojatnostima da je imao dosta visoko parking, ali zašto je odredio radno-vrijeme nije mi jasno</span>
<br>

#### Greška broj: 5
Krivo klasificirana poruka: Kolika je cijena članstva?
<br>
Predikcija: ulaznice
<br>
Točna vrijednost: clanstvo
<br>
Vjerojatnosti:  {'adresa': 0.09481911561811372, 'clanstvo': 0.0822699504393754, 'danas_izlozbe': 0.09156844517428195, 'kafic': 0.07897373240985758, 'kontakt': 0.08271213211370253, 'parking': 0.07992058717854597, 'pristupacnost': 0.07961014108870255, 'radno_vrijeme': 0.1378304083789733, 'toaleti': 0.09656805469477563, 'ulaznice': 0.1757274329036714}
<br>
#### Objašnjenje:
<span style="color:green">Riječ cijena nemamo u train skupu za oznaku "članstvo", nego samo za "ulaznice", pa je model vjerojatno mislio ovo je lagano vidim odmah po cijeni, tako da bi bilo dobro dodati rečenicu sa cijenom i za ostale oznake gdi bi se moglo spominjati cijena</span>
<br>


### Kako biste, na temelju rezultata testiranja, mogli poboljšati rad chatbota?
Mislim da bi prvo trebalo povećati Train skup za model, neke prijedloge sam napisao već, ali definitivno sinoninime, padeže, uvesti greške... uglavnom puno više primjera
<br>
<br>

### Koja je, prema vama, najslabija točka chatbota i zašto?
Mali train skup najveća mana, a sljedeće je možda bi se model mogao poboljšati